In [1]:
import os

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import Window

execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.1.1
      /_/

Using Python version 2.7.12 (default, Nov 19 2016 06:48:10)
SparkSession available as 'spark'.


In [2]:
sparkSession = SparkSession.builder.enableHiveSupport().master("local [2]").getOrCreate()

data = sparkSession.read.parquet("/data/sample264")
meta = sparkSession.read.parquet("/data/meta")

In [3]:
window1 = Window.partitionBy('userId').orderBy(col('count').desc())
window2 = Window.partitionBy('userId')

userToArtist = data.groupBy(col("userId"), col("artistId")) \
                   .count() \
                   .withColumn("rank", rank().over(window1)) \
                   .filter(col("rank") <= 100) \
                   .withColumn('sum_count', sum(col('count')).over(window2)) \
                   .withColumn('norm_count', col('count') / col('sum_count') * 0.5) \
                   .orderBy(desc("norm_count"), asc("userId"), asc("artistId")) \
                   .limit(40) \
                   .select(col("userId"), col("artistId")) \
                   .collect()

In [4]:
for val in userToArtist:
    print "%s %s" % val

66 993426
116 974937
128 1003021
131 983068
195 997265
215 991696
235 990642
288 1000564
300 1003362
321 986172
328 967986
333 1000416
346 982037
356 974846
374 1003167
428 993161
431 969340
445 970387
488 970525
542 969751
612 987351
617 970240
649 973851
658 973232
662 975279
698 995788
708 968848
746 972032
747 972032
776 997265
784 969853
806 995126
811 996436
837 989262
901 988199
923 977066
934 990860
957 991171
989 975339
999 968823
